# Funzioni d'utilità

In [1]:
import os
import shutil
import json
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
def remove_and_make_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        os.mkdir(path)
    else:
        os.mkdir(path)

In [3]:
def reformat_json(path):
    with open(path, "wr") as f:
        tmp = json.loads(f.read())
        json_obj = json.dumps(tmp, indent=4)
        f.write(json_obj)
        f.close()

In [4]:
def jaccard(set1, set2):
    intersection_size = len(set1 & set2)
    union_size = len(set1 | set2)
    if union_size > 0:
        return intersection_size / union_size
    else:
        return 0

In [5]:
import os
from itertools import combinations
import pandas as pd
import json

In [6]:
def extract_schema_paolo(directory, name, final_schema_directory, threshold, format, head):

    result_jaccard = {}
    for filename in os.listdir(directory):

        print(filename)
        file = os.path.join(directory, filename)
        df = pd.read_csv(file, encoding="utf-8", low_memory=False)
        df = df.head(head)
        
        columns = list(df.columns)
        n_cols = len(columns)
        combs = combinations(range(n_cols), 2)
        print("Numbero di confronti: ", n_cols * (n_cols-1) // 2)
        k = 0
        for c1, c2 in combs:
            
            set1 = set(df.iloc[:, c1])
            set2 = set(df.iloc[:, c2])

            s = jaccard(set1, set2)
            
            cc1 = columns[c1]
            cc2 = columns[c2]
            print(k, cc1, cc2, s)
            k += 1
            
            if s > threshold:
                result_jaccard.setdefault(cc1, set()).add(cc2)
                result_jaccard.setdefault(cc2, set()).add(cc1)
        
        del df

    s = set()
    for main_token, tokens in result_jaccard.items():
        tmp = set()
        tmp.add(main_token)
        tmp.update(tokens)
        s.add(frozenset(tmp))

    i = 0
    dict_synonyms = {}
    for tokens in s:
        tokens = list(tokens)
        tmp = tokens[0].split("-")[1]
        if tmp not in dict_synonyms.keys():
            dict_synonyms[tmp] = tokens
        else:
            dict_synonyms[tmp + "-" + str(i)] = tokens
            i += 1

    json_obj = json.dumps(dict_synonyms, indent=4)
    f = open(final_schema_directory + name + format, "w")
    f.write(json_obj)
    f.close()

    print("\n\n")

# Schemi mediati cluster

In [ ]:
from task_1 import make_cluster_columns_cluster
#from task_2 import schema_cluster
from task_4 import make_schema_cluster

In [ ]:
clusters_datasets_base_path = "..\\..\\Dataset\\ClusterParsed\\"
clusters_info_path = "..\\DatasetSchemaMatch\\"
clusters_filename_synonym = "dic_pre_val.txt"

clusters = ".\\clusters\\"
clusters_columns = ".\\clusters\\columns\\"
clusters_dictionary = ".\\clusters\\dictionary\\"
clusters_final_dictionary = ".\\clusters\\final_dictionary\\"
clusters_schema = ".\\clusters\\schema\\"

In [ ]:
#remove_and_make_dir(clusters)

### TASK 1

In [ ]:
print("\nMake cluster columns\n")
remove_and_make_dir(clusters_columns)
make_cluster_columns_cluster(clusters_datasets_base_path, clusters_info_path, clusters_filename_synonym, clusters_columns)

### TASK 2

In [ ]:
def schema_cluster(directory, final_schema_directory, threshold, format, head):
    for dirname in os.listdir(directory):
        print(dirname)
        cluster = os.path.join(directory, dirname)
        extract_schema_paolo(cluster, dirname, final_schema_directory, threshold, format, head)

In [ ]:
print("\nExtract schema\n")
remove_and_make_dir(clusters_dictionary)
start = time.time()
schema_cluster(clusters_columns, clusters_dictionary, 0.1, ".txt", 10000)
end = time.time()
print("\nTempo di esecuzione: "+str(end-start)+" (s)"+"\n")

### TASK 4

In [ ]:
print("\nMake schemas\n")
remove_and_make_dir(clusters_schema)
make_schema_cluster(clusters_datasets_base_path, clusters_final_dictionary, clusters_schema, ".txt")

# Schema mediato finale

In [ ]:
from task_1 import make_cluster_columns_final
#from task_2 import schema_final
from task_4 import make_schema_final

In [7]:
final_datasets_base_path = ".\\clusters\\schema\\"
final_info_path = ".\\files_matching\\files_vari"
final_filename_synonym = "dic_pre_val.txt"

final_schema = ".\\final_schema\\"
final_schema_columns = ".\\final_schema\\columns\\"
final_schema_dictionary = ".\\final_schema\\dictionary\\"
final_schema_final_dictionary = ".\\final_schema\\final_dictionary\\"
final_schema_schema = ".\\final_schema\\schema\\"

In [ ]:
#remove_and_make_dir(final_schema)

### TASK 1

In [ ]:
print("\nMake cluster columns\n")
remove_and_make_dir(final_schema_columns)
make_cluster_columns_final(final_datasets_base_path, final_info_path, final_filename_synonym, final_schema_columns)

### TASK 2

In [8]:
def schema_final(directory, final_schema_directory, threshold, format, head):
    extract_schema_paolo(directory, "finalschema", final_schema_directory, threshold, format, head)

In [9]:
print("\nExtract schema\n")
remove_and_make_dir(final_schema_dictionary)
start = time.time()
schema_final(final_schema_columns, final_schema_dictionary, 0.1, ".txt", 25000)
end = time.time()
print("\nTempo di esecuzione: "+str(end-start)+" (s)"+"\n")


Extract schema

address.csv
Numbero di confronti:  6
0 infoclipper-address_name globaldata-address 0.0001238083446824316
1 infoclipper-address_name hithorizons-address 0.0002990430622009569
2 infoclipper-address_name govuk-office_address 0.00024789291026276647
3 globaldata-address hithorizons-address 0.0001699524133242692
4 globaldata-address govuk-office_address 0.000152114390021296
5 hithorizons-address govuk-office_address 0.0005431830526887561
categories.csv
Numbero di confronti:  136
0 wikipedia-notes disfold-sector 0.003682272488164124
1 wikipedia-notes ft-sector 0.009404388714733543
2 wikipedia-notes valuetoday-sector 0.0005296610169491525
3 wikipedia-notes wikipedia-sector 0.04441190824792582
4 wikipedia-notes ambitiobox-industry 0.01586489252814739
5 wikipedia-notes ariregister-industry 0.0
6 wikipedia-notes cbinsights-industry 0.0005243838489774515
7 wikipedia-notes disfold-industry 0.012278978388998035
8 wikipedia-notes forbes-industry 0.007280291211648466
9 wikipedia-notes

132 valuetoday-industry disfold-categories 0.00047732696897374703
133 wikipedia-industry companiesmarketcap-categories 0.00042122999157540015
134 wikipedia-industry disfold-categories 0.001272264631043257
135 companiesmarketcap-categories disfold-categories 0.0003951007506914263
ceo.csv
Numbero di confronti:  15
0 ambitiobox-headquarter ariregister-headquarter 0.0
1 ambitiobox-headquarter disfold-official_name 0.0005094243504839531
2 ambitiobox-headquarter disfold-ceo 8.109642364771714e-05
3 ambitiobox-headquarter forbes-ceo 0.0002642706131078224
4 ambitiobox-headquarter valuetoday-ceo 0.0002733734281027884
5 ariregister-headquarter disfold-official_name 0.0
6 ariregister-headquarter disfold-ceo 0.0
7 ariregister-headquarter forbes-ceo 0.0
8 ariregister-headquarter valuetoday-ceo 0.0
9 disfold-official_name disfold-ceo 9.415309292910272e-05
10 disfold-official_name forbes-ceo 0.00048216007714561236
11 disfold-official_name valuetoday-ceo 0.000513347022587269
12 disfold-ceo forbes-ceo 0

83 infoclipper-country wikipedia-headquarters 0.0007692307692307692
84 valuetoday-country disfold-headquarters_continent 0.16470588235294117
85 valuetoday-country valuetoday-headquarters_continent 0.008771929824561403
86 valuetoday-country disfold-headquarters_country 0.006036217303822937
87 valuetoday-country infoclipper-state 0.01948051948051948
88 valuetoday-country disfold-headquarters 0.00027218290691344586
89 valuetoday-country globaldata-headquarters 0.5118110236220472
90 valuetoday-country teamblind-headquarters 0.0019646365422396855
91 valuetoday-country wikipedia-headquarters 0.0028551034975017845
92 disfold-headquarters_continent valuetoday-headquarters_continent 0.010638297872340425
93 disfold-headquarters_continent disfold-headquarters_country 0.008403361344537815
94 disfold-headquarters_continent infoclipper-state 0.007352941176470588
95 disfold-headquarters_continent disfold-headquarters 0.0002736726874657909
96 disfold-headquarters_continent globaldata-headquarters 0.19

23 forbes-founded ariregister-foundation_year 0.0
24 forbes-founded ft-founding_year 0.0
25 forbes-founded valuetoday-founders 0.001658374792703151
26 forbes-founded hithorizons-est_of_ownership 0.0
27 teamblind-founded valuetoday-founded 0.006944444444444444
28 teamblind-founded wikipedia-founded 0.45454545454545453
29 teamblind-founded ambitiobox-foundation_year 0.0
30 teamblind-founded ariregister-foundation_year 0.0
31 teamblind-founded ft-founding_year 0.0
32 teamblind-founded valuetoday-founders 0.0018867924528301887
33 teamblind-founded hithorizons-est_of_ownership 0.0
34 valuetoday-founded wikipedia-founded 0.003289473684210526
35 valuetoday-founded ambitiobox-foundation_year 0.0
36 valuetoday-founded ariregister-foundation_year 0.0
37 valuetoday-founded ft-founding_year 0.0
38 valuetoday-founded valuetoday-founders 0.002583979328165375
39 valuetoday-founded hithorizons-est_of_ownership 0.0
40 wikipedia-founded ambitiobox-foundation_year 0.0
41 wikipedia-founded ariregister-fou

Numbero di confronti:  300
0 infoclipper-trade_name ambitiobox-industry 0.0019193857965451055
1 infoclipper-trade_name ariregister-industry 0.0
2 infoclipper-trade_name cbinsights-industry 0.0022522522522522522
3 infoclipper-trade_name disfold-industry 0.0016750418760469012
4 infoclipper-trade_name forbes-industry 0.0021141649048625794
5 infoclipper-trade_name globaldata-industry 0.002197802197802198
6 infoclipper-trade_name hithorizons-industry 0.002304147465437788
7 infoclipper-trade_name teamblind-industry 0.002012072434607646
8 infoclipper-trade_name valuetoday-industry 0.00024131274131274132
9 infoclipper-trade_name wikipedia-industry 0.0013531799729364006
10 infoclipper-trade_name ambitiobox-name 0.00010497585555322275
11 infoclipper-trade_name ariregister-name 0.0005282620179609086
12 infoclipper-trade_name cbinsights-name 0.0006127450980392157
13 infoclipper-trade_name companiesmarketcap-name 0.0
14 infoclipper-trade_name disfold-name 5.9883825378765195e-05
15 infoclipper-trade

149 hithorizons-industry wikipedia-industry 0.009316770186335404
150 hithorizons-industry ambitiobox-name 0.0001097574360662935
151 hithorizons-industry ariregister-name 0.0006765899864682003
152 hithorizons-industry cbinsights-name 0.0008216926869350862
153 hithorizons-industry companiesmarketcap-name 0.0
154 hithorizons-industry disfold-name 6.140997297961189e-05
155 hithorizons-industry forbes-name 0.00048449612403100775
156 hithorizons-industry ft-name 0.0002403268445085316
157 hithorizons-industry globaldata-name 0.00018709073900841907
158 hithorizons-industry govuk-name 0.0001452643811737362
159 hithorizons-industry hithorizons-name 0.0014771048744460858
160 hithorizons-industry infoclipper-name 0.0003161555485298767
161 hithorizons-industry teamblind-name 0.0010460251046025104
162 hithorizons-industry valuetoday-name 0.0
163 hithorizons-industry wikipedia-name 0.00027329871549603714
164 teamblind-industry valuetoday-industry 0.0023784355179704017
165 teamblind-industry wikipedia

3 companiesmarketcap-share_price globaldata-market_cap 0.00010424267695194412
4 companiesmarketcap-share_price valuetoday-annual_net_income_usd 0.00016335865392469166
5 companiesmarketcap-share_price companiesmarketcap-revenue 0.00012127091923356779
6 companiesmarketcap-share_price disfold-revenue 0.00011349449551696743
7 companiesmarketcap-share_price globaldata-revenue 0.00011235955056179776
8 companiesmarketcap-share_price valuetoday-revenue 0.00012127091923356779
9 companiesmarketcap-share_price forbes-sales 9.74184120798831e-05
10 companiesmarketcap-share_price companiesmarketcap-earnings 0.00012127091923356779
11 companiesmarketcap-share_price forbes-market_value 0.00010081661457808247
12 forbes-profit companiesmarketcap-pricecap 0.06459731543624161
13 forbes-profit disfold-market_cap 0.05621669626998224
14 forbes-profit globaldata-market_cap 0.029356417011667295
15 forbes-profit valuetoday-annual_net_income_usd 0.0591937069813176
16 forbes-profit companiesmarketcap-revenue 0.000

77 valuetoday-annual_net_income_usd disfold-revenue 0.00043830813061582295
78 valuetoday-annual_net_income_usd globaldata-revenue 0.011299435028248588
79 valuetoday-annual_net_income_usd valuetoday-revenue 0.00025006251562890725
80 valuetoday-annual_net_income_usd forbes-sales 0.054115586690017516
81 valuetoday-annual_net_income_usd companiesmarketcap-earnings 0.00025006251562890725
82 valuetoday-annual_net_income_usd forbes-market_value 0.04513633014001474
83 valuetoday-annual_net_income_usd valuetoday-annual_revenue_usd 0.2342902350813743
84 ft-revenue_2017_eu companiesmarketcap-revenue 0.0005002501250625312
85 ft-revenue_2017_eu disfold-revenue 0.000390015600624025
86 ft-revenue_2017_eu globaldata-revenue 0.0003769317753486619
87 ft-revenue_2017_eu valuetoday-revenue 0.0005002501250625312
88 ft-revenue_2017_eu forbes-sales 0.00024888003982080636
89 ft-revenue_2017_eu companiesmarketcap-earnings 0.0005002501250625312
90 ft-revenue_2017_eu forbes-market_value 0.0002723311546840959
91 

### TASK 4

In [ ]:
print("\nMake schema\n")
remove_and_make_dir(final_schema_schema)
make_schema_final(final_datasets_base_path, final_schema_final_dictionary, final_schema_schema, ".json")